<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/YOLOv5-LIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**YOLOv5 LIME CorneAI**

##**Setup YOLOv5**

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd /content
!pip uninstall deep_utils -y
!pip install -U git+https://github.com/pooya-mohammadi/deep_utils.git --q
!pip install torch --q
!pip install torchvision --q
!pip install -U opencv-python --q
print("[INFO] To use new installed version of opencv, the session should be restarted!!!!")

!git clone https://github.com/pooya-mohammadi/yolov5-gradcam

/content
  Preparing metadata (setup.py) ... done
[INFO] To use new installed version of opencv, the session should be restarted!!!!
Cloning into 'yolov5-gradcam'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 134 (delta 29), reused 36 (delta 13), pack-reused 68 (from 1)
Receiving objects: 100% (134/134), 6.95 MiB | 17.31 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [3]:
import os
os.chdir('/content/yolov5-gradcam')

model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial/3.jpg"

#**LIME**

In [4]:
!pip install lime --q
!pip install scikit-image --q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
import cv2
import time
import traceback
import torchvision
from lime import lime_image
from deep_utils.utils.box_utils.boxes import Box
from models.experimental import attempt_load
from utils.general import xywh2xyxy
from utils.datasets import letterbox
from utils.metrics import box_iou
%matplotlib inline

# 角膜AIのクラス定義
CORNEA_CLASSES = [
    "infection",
    "normal",
    "non-infection",
    "scar",
    "tumor",
    "deposit",
    "APAC",
    "lens opacity",
    "bullous"
]

def setup_device():
    """
    GPUが利用可能な場合はGPUを、そうでない場合はCPUを設定

    Returns:
    --------
    device : torch.device
        使用するデバイス
    """
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
        print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    else:
        device = torch.device('cpu')
        print("GPU not available, using CPU")
    return device

class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size=(640, 640),
                 names=CORNEA_CLASSES,
                 mode='eval',
                 confidence=0.25,
                 iou_thresh=0.45,
                 agnostic_nms=False):
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.agnostic = agnostic_nms

        # モデルのロード
        print("[INFO] Loading cornea detection model...")
        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model loaded successfully")

        # モデルのクラス数を取得と確認
        self.nc = int(self.model.nc)
        print(f"[INFO] Number of classes: {self.nc}")

        # クラス名の設定と検証
        self.names = names
        if len(self.names) != self.nc:
            print(f"[WARNING] Number of class names ({len(self.names)}) does not match model classes ({self.nc})")
        print(f"[INFO] Using class names: {self.names}")

        self.model.requires_grad_(True)
        self.model.to(device)
        if self.mode == 'train':
            self.model.train()
        else:
            self.model.eval()

        # Cold start prevention
        print("[INFO] Performing cold start prevention...")
        img = torch.zeros((1, 3, *self.img_size), device=device)
        self.model(img)
        print("[INFO] Initialization complete")

    @staticmethod
    def non_max_suppression(prediction, logits, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False,
                            multi_label=False, labels=(), max_det=300):
        nc = prediction.shape[2] - 5
        xc = prediction[..., 4] > conf_thres

        # Checks
        assert 0 <= conf_thres <= 1, f'Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0'
        assert 0 <= iou_thres <= 1, f'Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0'

        # Settings
        min_wh, max_wh = 2, 4096
        max_nms = 30000
        time_limit = 10.0
        redundant = True
        multi_label &= nc > 1
        merge = False

        t = time.time()
        output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]
        logits_output = [torch.zeros((0, nc), device=logits.device)] * logits.shape[0]

        for xi, (x, log_) in enumerate(zip(prediction, logits)):
            x = x[xc[xi]]
            log_ = log_[xc[xi]]

            if not x.shape[0]:
                continue

            x[:, 5:] *= x[:, 4:5]
            box = xywh2xyxy(x[:, :4])

            if multi_label:
                i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
                x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
            else:
                conf, j = x[:, 5:].max(1, keepdim=True)
                x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]
                log_ = log_[conf.view(-1) > conf_thres]

            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

            n = x.shape[0]
            if not n:
                continue
            elif n > max_nms:
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]

            c = x[:, 5:6] * (0 if agnostic else max_wh)
            boxes, scores = x[:, :4] + c, x[:, 4]
            i = torchvision.ops.nms(boxes, scores, iou_thres)

            if i.shape[0] > max_det:
                i = i[:max_det]

            output[xi] = x[i]
            logits_output[xi] = log_[i]

            if (time.time() - t) > time_limit:
                print(f'WARNING: NMS time limit {time_limit}s exceeded')
                break

        return output, logits_output

    @staticmethod
    def yolo_resize(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):
        return letterbox(img, new_shape=new_shape, color=color, auto=auto, scaleFill=scaleFill, scaleup=scaleup)

    def forward(self, img):
        try:
            prediction, logits, _ = self.model(img, augment=False)
            prediction, logits = self.non_max_suppression(prediction, logits, self.confidence, self.iou_thresh,
                                                          classes=None,
                                                          agnostic=self.agnostic)

            batch_size = img.shape[0]
            self.boxes = [[] for _ in range(batch_size)]
            self.class_names = [[] for _ in range(batch_size)]
            self.classes = [[] for _ in range(batch_size)]
            self.confidences = [[] for _ in range(batch_size)]

            for i, det in enumerate(prediction):
                if len(det):
                    for *xyxy, conf, cls in det:
                        xyxy[0] = max(0, xyxy[0])
                        xyxy[1] = max(0, xyxy[1])
                        xyxy[2] = min(self.img_size[1], xyxy[2])
                        xyxy[3] = min(self.img_size[0], xyxy[3])

                        bbox = Box.box2box(xyxy,
                                           in_source=Box.BoxSource.Torch,
                                           to_source=Box.BoxSource.Numpy,
                                           return_int=True)

                        self.boxes[i].append(bbox)
                        self.confidences[i].append(float(conf.item()))
                        cls_idx = int(cls.item())

                        if cls_idx >= len(self.names):
                            print(f"[WARNING] Class index {cls_idx} is out of range")
                            cls_idx = 0

                        self.classes[i].append(cls_idx)
                        self.class_names[i].append(self.names[cls_idx])

            return [self.boxes, self.classes, self.class_names, self.confidences], logits

        except Exception as e:
            print(f"Error in forward pass: {e}")
            traceback.print_exc()
            return [[[]], [[]], [[]], [[]]], None

    def preprocessing(self, img):
        try:
            if len(img.shape) != 4:
                img = np.expand_dims(img, axis=0)
            im0 = img.astype(np.uint8)
            img = np.array([self.yolo_resize(im, new_shape=self.img_size)[0] for im in im0])
            img = img.transpose((0, 3, 1, 2))
            img = np.ascontiguousarray(img)
            img = torch.from_numpy(img).to(self.device)
            img = img / 255.0
            return img
        except Exception as e:
            print(f"Error in preprocessing: {e}")
            traceback.print_exc()
            return None

class YOLOLimeExplainer:
    def __init__(self, yolo_model, device='cuda', img_size=(640, 640)):
        self.model = yolo_model
        self.device = device
        self.img_size = img_size
        self.explainer = lime_image.LimeImageExplainer()

    def predict_fn(self, images):
        try:
            batch_predictions = []
            for img in images:
                processed_img = self.model.preprocessing(np.expand_dims(img, 0))
                if processed_img is None:
                    raise ValueError("Failed to preprocess image")

                with torch.no_grad():
                    predictions, _ = self.model(processed_img)

                class_scores = np.zeros(len(CORNEA_CLASSES))

                if predictions[0][0]:
                    for cls_idx, conf in zip(predictions[1][0], predictions[3][0]):
                        if cls_idx < len(CORNEA_CLASSES):
                            class_scores[cls_idx] = max(class_scores[cls_idx], conf)

                batch_predictions.append(class_scores)

            return np.array(batch_predictions)

        except Exception as e:
            print(f"Error in prediction_fn: {e}")
            traceback.print_exc()
            return np.zeros((len(images), len(CORNEA_CLASSES)))

    def explain_instance(self, image, num_samples=1000, top_labels=5):
        try:
            if isinstance(image, Image.Image):
                image = np.array(image)
            if len(image.shape) == 2:
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            elif image.shape[2] == 4:
                image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

            explanation = self.explainer.explain_instance(
                image,
                self.predict_fn,
                labels=range(len(CORNEA_CLASSES)),
                top_labels=top_labels,
                hide_color=0,
                num_samples=num_samples
            )
            return explanation

        except Exception as e:
            print(f"Error in explain_instance: {e}")
            traceback.print_exc()
            return None

def visualize_results(explanation, image, class_names=CORNEA_CLASSES, save_path=None):
    """
    Visualize LIME explanation results overlaid on the original image

    Args:
        explanation: LIME explanation object
        image: Original image array
        class_names: List of class names
        save_path: Optional path to save the visualization
    """
    try:
        # Get predictions
        prediction = explanation.predict_fn(np.array([image]))[0]

        # Sort labels by prediction confidence
        sorted_labels = sorted(range(len(prediction)),
                             key=lambda x: prediction[x],
                             reverse=True)[:2]

        # Create a figure with subplots
        plt.figure(figsize=(15, 7))

        for idx, label in enumerate(sorted_labels):
            plt.subplot(1, len(sorted_labels), idx + 1)

            # Get the original image and mask from LIME
            mask = explanation.local_exp[label]

            # Convert the sparse mask to a dense array
            dense_mask = np.zeros(explanation.segments.shape, dtype=float)
            for i, v in mask:
                dense_mask[explanation.segments == i] = v

            # Normalize the mask to [0, 1] range
            if dense_mask.max() != dense_mask.min():
                dense_mask = (dense_mask - dense_mask.min()) / (dense_mask.max() - dense_mask.min())

            # Create a colormap (red for positive contributions)
            heatmap = np.zeros((dense_mask.shape[0], dense_mask.shape[1], 4))
            heatmap[:, :, 2] = dense_mask  # Blue channel
            heatmap[:, :, 3] = dense_mask * 1.0  # Alpha channel

            # Display original image
            plt.imshow(image, alpha=0.8)

            # Overlay heatmap
            plt.imshow(heatmap, alpha=0.6)

            class_name = class_names[label] if label < len(class_names) else f"Unknown Class {label}"
            plt.title(f'{class_name}\nConfidence: {prediction[label]:.3f}', fontsize=12)
            plt.axis('off')

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=300)
            print(f"Visualization saved to {save_path}")

        plt.show()

        print("\nDetailed Confidence Scores:")
        for label in sorted_labels:
            print(f"{class_names[label]}: {prediction[label]:.3f}")

    except Exception as e:
        print(f"Error in visualization: {e}")
        traceback.print_exc()

class YOLOLimeExplainer:
    def __init__(self, yolo_model, device='cuda', img_size=(640, 640)):
        self.model = yolo_model
        self.device = device
        self.img_size = img_size
        self.explainer = lime_image.LimeImageExplainer()

    def predict_fn(self, images):
        """
        Modified prediction function with improved image processing
        """
        try:
            batch_predictions = []
            for img in images:
                # Normalize image if needed
                if img.max() > 1.0:
                    img = img / 255.0

                # Convert image format if needed
                if len(img.shape) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                elif img.shape[2] == 4:
                    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)

                # Ensure image is uint8 for preprocessing
                img_uint8 = (img * 255).astype(np.uint8)
                processed_img = self.model.preprocessing(np.expand_dims(img_uint8, 0))

                if processed_img is None:
                    raise ValueError("Failed to preprocess image")

                with torch.no_grad():
                    predictions, logits = self.model(processed_img)

                class_scores = np.zeros(len(CORNEA_CLASSES))

                if predictions[0][0]:  # If there are any detections
                    if logits is not None and len(logits[0]) > 0:
                        # Use logits if available
                        probs = torch.nn.functional.softmax(logits[0], dim=1)
                        class_scores = probs.mean(dim=0).cpu().numpy()
                    else:
                        # Fallback to confidence scores
                        for cls_idx, conf in zip(predictions[1][0], predictions[3][0]):
                            if cls_idx < len(CORNEA_CLASSES):
                                class_scores[cls_idx] = max(class_scores[cls_idx], conf)

                batch_predictions.append(class_scores)

            return np.array(batch_predictions)

        except Exception as e:
            print(f"Error in prediction_fn: {e}")
            traceback.print_exc()
            return np.zeros((len(images), len(CORNEA_CLASSES)))

    def explain_instance(self, image, num_samples=1000, top_labels=5):
        """
        Generate LIME explanation with improved image handling
        """
        try:
            # Convert PIL Image to numpy array if needed
            if isinstance(image, Image.Image):
                image = np.array(image)

            # Normalize image if needed
            if image.max() > 1.0:
                image = image.astype(float) / 255.0

            # Convert image format if needed
            if len(image.shape) == 2:
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            elif image.shape[2] == 4:
                image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

            # Generate explanation
            explanation = self.explainer.explain_instance(
                image,
                self.predict_fn,
                labels=range(len(CORNEA_CLASSES)),
                top_labels=top_labels,
                hide_color=0,
                num_samples=num_samples
            )

            # Store predict_fn for later use
            explanation.predict_fn = self.predict_fn

            return explanation

        except Exception as e:
            print(f"Error in explain_instance: {e}")
            traceback.print_exc()
            return None

def run_lime_analysis(model_path, img_path, num_samples=500, save_path=None):
    """
    Run LIME analysis with improved visualization

    Args:
        model_path: Path to the YOLO model weights
        img_path: Path to the input image
        num_samples: Number of samples for LIME analysis
        save_path: Optional path to save the visualization
    """
    try:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")

        print("Loading YOLO model...")
        yolo_model = YOLOV5TorchObjectDetector(
            model_weight=model_path,
            device=device,
            img_size=(640, 640),
            names=CORNEA_CLASSES
        )

        print("Initializing LIME explainer...")
        lime_explainer = YOLOLimeExplainer(yolo_model)

        print("Loading and processing image...")
        image = Image.open(img_path)
        image_array = np.array(image)

        # Normalize image if needed
        if image_array.max() > 1.0:
            image_array = image_array.astype(float) / 255.0

        print(f"Running LIME analysis with {num_samples} samples...")
        explanation = lime_explainer.explain_instance(
            image_array,
            num_samples=num_samples,
            top_labels=3
        )

        if explanation is None:
            print("Failed to generate explanation")
            return None

        print("Visualizing results...")
        visualize_results(explanation, image_array, save_path=save_path)

        return explanation

    except Exception as e:
        print(f"Error in run_lime_analysis: {e}")
        traceback.print_exc()
        return None

# Usage example:
if __name__ == "__main__":
    model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial/1.jpg"
    save_path = "lime_explanation.png"  # Optional

    explanation = run_lime_analysis(
        model_path=model_path,
        img_path=img_path,
        num_samples=500,
        save_path=save_path
    )